# How to implement a new Workflow

The Transformation Framework can be extended by adding custom Workflows.

Each Workflow shall be contained in a python package that:

1. Implement the workflow processing function that executes the processing
2. Define in a dictionary the Workflow Configuration, which will include the reference to the workflow processing function and the description of the input parameters.
3. Expose the Workflow Configuration as an entrypoint in the package setup


## 1. Workflow processing function

The python package shall implement the workflow processing function that takes in input:

- the product path
- the workflow options, a dictionary containing the input parameters
- the processing directory

It shall return the path of the output product.


In [ ]:
import typing as T

def run_processing(
    product_path: str,
    *,
    workflow_options: T.Dict[str, T.Union[str, int, float, bool]],
    processing_dir: str,
   )-> str:
    """
    Execute the processing and return the path of the output product.

    :param str product_path: path of the main Sentinel-2 L1C product folder
    :param dict workflow_options: the user's options dictionary
    :param str processing_dir: path of the processing directory
    :return str:
    """

    ...
    
    output_product_path = ...
    return output_product_path


## 2. Workflow Configuration Dictionary

The Workflow Configuration is a Python dictionary defined in the workflow package that contains the reference to the workflow processing function and the description of the workflow input parameters.

#### Example: Sen2Cor workflow definition
The following example is [Sen2Cor workflow definition](https://github.com/DHS-TransformationFramework/esa_tf/blob/a71f9c22f2b87e9298e10c07abdd022b807f8e08/esa_tf_platform/esa_tf_platform/esa_tf_plugin_sen2cor.py#L290):

In [ ]:
sen2cor_l1c_l2a = {
    "WorkflowName": "Sen2Cor_L1C_L2A",
    "Description": "Product processing from Sentinel-2 L1C to L2A using Sen2Cor v2.10, supporting Level-1C product version 14.2 - 14.6",
    "Execute": "esa_tf_platform.esa_tf_plugin_sen2cor.run_processing",
    "InputProductType": "S2MSI1C",
    "OutputProductType": "S2MSI2A",
    "WorkflowVersion": "0.1",
    "WorkflowOptions": {
        "Aerosol_Type": {
            "Description": "Default processing via configuration is the rural (continental) aerosol type with mid latitude summer and an ozone concentration of 331 Dobson Units",
            "Type": "string",
            "Default": "RURAL",
            "Enum": ["MARITIME", "RURAL"],
        },
        "Mid_Latitude": {
            "Description": "If  'AUTO' the atmosphere profile will be determined automatically by the processor, selecting WINTER or SUMMER atmosphere profile based on the acquisition date and geographic location of the tile",
            "Type": "string",
            "Default": "SUMMER",
            "Enum": ["SUMMER", "WINTER", "AUTO"],
        },
        "Ozone_Content": {
            "Description": "0: to get the best approximation from metadata (this is the smallest difference between metadata and column DU), else select for midlatitude summer (MS) atmosphere: 250, 290, 331 (standard MS), 370, 410, 450; for midlatitude winter (MW) atmosphere: 250, 290, 330, 377 (standard MW), 420, 460",
            "Type": "integer",
            "Default": 331,
            "Enum": [0, 250, 290, 330, 331, 370, 377, 410, 420, 450, 460],
        },
        "Cirrus_Correction": {
            "Description": "FALSE: no cirrus correction applied, TRUE: cirrus correction applied",
            "Type": "boolean",
            "Default": False,
        },
        "DEM_Terrain_Correction": {
            "Description": "Use DEM for Terrain Correction, otherwise only used for WVP and AOT",
            "Type": "boolean",
            "Default": True,
        },
        "Resolution": {
            "Description": "Target resolution, can be 10, 20 or 60m. If omitted, 10, 20 and 60m resolutions will be processed",
            "Type": "integer",
            "Enum": [10, 20, 60],
            "Default": None,
        },
    },
}

#### Workflow Configuration dictionary structure

The following set of keys are mandatory:  

- "WorkflowName" is the name of the Workflow
- "Description" is a brief description of the workflow
- "Execute" is a string containing the reference to the Python function that executes the workflow
- "InputProductType" is a valid Sentinel-1 or Sentinel-2 product type
- "OutputProductType" is the output product type
- "WorkflowVersion" is the version of the workflow
- "WorkflowOptions" is a dictionary containing the definition of the workflow input options


The keys of "WorkflowOptions" shall be the input parameter names, the values are dictionaries describing the parameters:
- "Description” (mandatory) 
- "Type" (mandatory)
- “Default” (optional)
- “Enum” (list of the allowed values, optional)

## 3. Workflow configuration entrypoint


In order to allow Transformation Framework to detect the workflow, is necessary to register the Workflow Configuration.

The registration of the workflow is performed via the entry-point system. 
The Workflow Configuration dictionary shall be registered in the setup of the workflow package, as follows:
- the group shall be "esa_tf.plugin"
- the name shall be the workflow unique identifier
- the object reference shall be the Workflow Configuration dictionary reference

 #### Example: [Sen2Cor workflow definition](https://github.com/DHS-TransformationFramework/esa_tf/blob/a71f9c22f2b87e9298e10c07abdd022b807f8e08/esa_tf_platform/setup.cfg#L33) in setup.cfg 

In [ ]:
entry_points=""" 
    [esa_tf.plugin] 
    sen2cor_l1c_l2a = esa_tf_platform.esa_tf_plugin_sen2cor:sen2cor_l1c_l2a
"""